# 基于机器学习数据库飞速上线AI应用——RUL

剩余使用寿命（remaining useful life，RUL），指一个系统正常工作一段时间后,能够正常运转的时间。借助RUL,工程师可以安排维护时间、优化运行效率并避免计划外停机。因此,预测RUL是预测性维护计划中的首要任务。 
本次的任务就是开发一个通过机器学习模型进行剩余使用寿命预测的实时智能应用。我们使用NASA提供的[Turbofan Engine Degradation Simulation Data Set](https://ti.arc.nasa.gov/tech/dash/groups/pcoe/prognostic-data-repository/#turbofan)，作为训练集与测试集。

整个应用开发是基于[notebook](http://ipython.org/notebook.html)。


## 初始化环境
整个初始化过程包含安装fedb，以及相关运行环境，初始化脚步可以参考https://github.com/4paradigm/DemoApps/blob/main/predict-remaining-useful-life-nb/demo/init.sh

In [1]:
!cd demo && sh init.sh

ZooKeeper JMX enabled by default
Using config: /home/jovyan/work/zookeeper-3.4.14/bin/../conf/zoo.cfg
Starting zookeeper ... STARTED
Starting tablet ... STARTED
Starting nameserver ... STARTED
2021-06-18 14:27:23,989:899(0x7f96184eaa00):ZOO_INFO@log_env@753: Client environment:zookeeper.version=zookeeper C client 3.4.14
2021-06-18 14:27:23,990:899(0x7f96184eaa00):ZOO_INFO@log_env@757: Client environment:host.name=m7-pce-dev01
2021-06-18 14:27:23,990:899(0x7f96184eaa00):ZOO_INFO@log_env@764: Client environment:os.name=Linux
2021-06-18 14:27:23,990:899(0x7f96184eaa00):ZOO_INFO@log_env@765: Client environment:os.arch=3.10.0-1127.18.2.el7.x86_64
2021-06-18 14:27:23,990:899(0x7f96184eaa00):ZOO_INFO@log_env@766: Client environment:os.version=#1 SMP Sun Jul 26 15:27:06 UTC 2020
2021-06-18 14:27:23,990:899(0x7f96184eaa00):ZOO_INFO@log_env@774: Client environment:user.name=(null)
2021-06-18 14:27:23,990:899(0x7f96184eaa00):ZOO_INFO@log_env@782: Client environment:user.home=/root
2021-06-18 14:2

## 导入历史数据到fedb

使用fedb进行时序特征计算是需要历史数据的，所以我们将历史数据导入到fedb，以便实时推理可以使用历史数据进行特征推理，导入代码可以参考https://github.com/4paradigm/DemoApps/blob/main/predict-taxi-trip-duration-nb/demo/import.py 。

这里使用`test_FD004.txt`作为历史数据，有多个engine多次cycle的检测数据。

In [2]:
!cd demo && python3 import.py

2021-06-18 14:27:25,305:914(0x7f3792762740):ZOO_INFO@log_env@753: Client environment:zookeeper.version=zookeeper C client 3.4.14
2021-06-18 14:27:25,305:914(0x7f3792762740):ZOO_INFO@log_env@757: Client environment:host.name=m7-pce-dev01
2021-06-18 14:27:25,305:914(0x7f3792762740):ZOO_INFO@log_env@764: Client environment:os.name=Linux
2021-06-18 14:27:25,305:914(0x7f3792762740):ZOO_INFO@log_env@765: Client environment:os.arch=3.10.0-1127.18.2.el7.x86_64
2021-06-18 14:27:25,305:914(0x7f3792762740):ZOO_INFO@log_env@766: Client environment:os.version=#1 SMP Sun Jul 26 15:27:06 UTC 2020
2021-06-18 14:27:25,305:914(0x7f3792762740):ZOO_INFO@log_env@774: Client environment:user.name=(null)
2021-06-18 14:27:25,305:914(0x7f3792762740):ZOO_INFO@log_env@782: Client environment:user.home=/root
2021-06-18 14:27:25,305:914(0x7f3792762740):ZOO_INFO@log_env@794: Client environment:user.dir=/home/jovyan/work/rul/demo
2021-06-18 14:27:25,305:914(0x7f3792762740):ZOO_INFO@zookeeper_init@827: Initiating cli

## 生成训练用的特征矩阵
模型训练需要训练数据，以下是生成使用的的代码

* 训练特征矩阵生成脚本代码 https://github.com/4paradigm/DemoApps/blob/main/predict-remaining-useful-life-nb/demo/gene_train_by_spark.py
* 训练数据使用`train_FD004.txt`，多engine不同cycle的检测数据。

整个任务最终会生成训练用的特征矩阵，保存为train_fm.csv。

### 特征详解
特征可分为两类，一类是一个engine的聚合特征，用sql可以表达为
```
select engine_no, MAX(operational_setting_1),... from d2 group by engine_no;
```
二类是相同time_in_cycles的所有engine的共同聚合特征，用sql可以表达为
```
select time_in_cycles, MAX(operational_setting_1) as rcm_operational_setting_1, ... from d2 group by time_in_cycles;""")

```


In [3]:
!cd demo && sh train_fm.sh

21/06/18 14:27:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/06/18 14:27:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
Loaded data with:
61249 Recordings
249 Engines
21 Sensor Measurements
3 Operational Settings
Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|████████| engine_no: 249/249 
spark:
21/06/18 14:27:49 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
I0618 14:27:53.574008  1117 sql_compiler.cc:61] keep ir length: 68353
    PROJECT(type=WindowAggregation)
      +-WINDOW(partition_keys=(engine_no), orders=(record_time) ASC, range=(recor

## 使用训练的模型搭建链接fedb的实时推理http服务

基于上一步生成的训练用特征矩阵，使用`RandomForestRegressor`进行训练。
得到的训练模型，再结合fedb中的历史数据，搭建一个实时推理服务，整个推理服务代码参考https://github.com/4paradigm/DemoApps/blob/main/predict-remaining-useful-life-nb/demo/predict_server.py 。

In [2]:
!cd demo && sh start_predict_server.sh
import os
output = os.popen('ps -ef | grep predict && kill -9 4791')
# output = os.popen('cat /tmp/p.log')
print(output.read())
# !cd demo && python3 predict_server.py

start predict server
root      1591     1 99 14:36 ?        00:00:05 python3 predict_server.py
root      1634  1575  0 14:36 ?        00:00:00 /bin/sh -c ps -ef | grep predict && kill -9 4791
root      1636  1634  0 14:36 ?        00:00:00 grep predict



## 通过http请求发送一个推理请求

In [3]:
!cd demo && python3 predict.py

----------------ins---------------
        1      2      3       4    5      6       7        8        9        10    11     12      13     14      15     16       17       18       19     20     21     22       23       24       25    26     27      28   29     30      31       32       33      34    35     36      37     38      39     40       41       42       43     44     45      46       47       48
0  42.0075  0.842  100.0  518.67  1.3  47.28  520.24  2388.03  8128.59  10.9106  0.03  393.0  2388.0  100.0  642.35  39.06  23.4425  1584.67  1404.36  14.62  21.57  552.9  2387.94  9042.66  35.0033  0.84  100.0  518.67  1.3  47.14  519.09  2387.42  8116.94  8.4244  0.03  392.0  2388.0  100.0  642.02  38.75  23.4425  1576.87  1400.86  14.62  21.57  551.79  2387.35  9032.46
---------------predict rul -------------
[240.08]
Congraduation! You have finished the task.
Your Key:b'o9NgmB0PK/cb2aG1ByKPMCXh/1TS4C03f/MrvTXQvSFY+9B1xMOvvgwkfEY='
